# Network reductions

Currently, there are two ways in which you can reduce a Boolean network: syntactically, and symbolically.

### Syntactic reduction

The syntactic case is easier to follow, but also less scalable. For very large networks, the function expressions can grow exponentially, which makes it impossible to perform without additional simplification.

In [1]:
from biodivine_aeon import *
import biodivine_aeon as ba
ba.LOG_LEVEL = ba.LOG_NOTHING # For this tutorial, we can turn off log messages since most computations will be very simple.

In [2]:
bn = BooleanNetwork.from_bnet("""
a, c | b
b, c & !a
c, a | b 
""")

In [3]:
bn2 = bn.inline_variable("c")
print(bn2.to_bnet())

targets,factors
a, ((a | b) | b)
b, ((a | b) & !a)



Note that you can't inline a variable if its update function depends on itself. 

 > This check is performed syntactically. So if `x` has the update function `(x & !x) | y`, this still counts as a self-regulation even though `x` is semantically not essential in the update function. This is not an issue in the symbolic reduction that is shown later.

In [4]:
try:
    bn2.inline_variable("a")
except RuntimeError as e:
    print(e)

Variable has a self-regulation.


### Symbolic reduction

We can perform the same process, but using a symbolic representation of the network (i.e. the `AsynchronousGraph`). 

 > Do note that symbolic representations of the original and the reduced network use the same names for variables, but are otherwise not compatible (e.g. `VariableId(5)` can reference a different variable in the original and in the reduced network). However, as we show a bit later, there is a translation method that can "move" sets between such compatible representations.

In [5]:
graph = AsynchronousGraph(bn)

In [6]:
graph2 = graph.inline_variable_symbolic("c")

In [7]:
# Here, the reduced network is *reconstructed* from the symbolic BDD representation.
# Once many reductions are applied, this typically produces a result that is smaller than what one
# would obtain using the syntactic reduction, but in some extreme cases even this can produce a
# network that is too large to reconstruct (e.g. model ID 122 in BBM).
bn2_reconstructed = graph2.reconstruct_network()
print(bn2_reconstructed.to_bnet())

targets,factors
a, (b | a)
b, (!a & b)



### Reduction and parameters

In AEON, Boolean networks can contain parameters (uninterpreted functions) that designate some unknown or partially known component of the network dynamics. Fundamentally, these should be handled by the reductions without any issues. However, do note that there is one difference between the syntactic and the symbolic reduction.

During the syntactic inlining, the information about interaction monotonicity and essentiality can be lost. For example, assume I have a variable `z` whose update function is `f(x, y)` (`f` being a parameter). Then, assuming that both `x` and `y` are essential activators, `f` can be only instantiated as `x & y` or `x | y`. However, assume that the update function of `x` is `!y`, and I try to eliminate `x`. Then the essentiality constraint can be retained, but `y` is no longer an activator, but rather a mixed/dual regulator. Hence `f(!y, y)` can be suddenly also instantiated as `y <=> !y`, `y ^ !y`, or many other functions.

Such "lost" static constraints can in theory be taken into account by only using the original subset of colors (function interpretations) when subsequently working with the symbolic representation of the reduced network.

In [8]:
bn = BooleanNetwork.from_aeon("""
x -> z
y -> z
$z: f(x,y)
y -| x
$x: !y
$y: true
""")

In [9]:
# With the static constraints, `f` can be only a conjunction or a disjunction.
graph = AsynchronousGraph(bn)
for x in graph.mk_unit_colors():
    print(x)

ColorModel({'f': '(x_0 & x_1)'})
ColorModel({'f': '((!x_0 & x_1) | x_0)'})


In [10]:
bn2 = bn.inline_variable("x")
print(bn2.to_aeon())

y -? z
$y: true
$z: f(!y, y)



In [11]:
# Suddenly, `f` can be many things, because the only requirement is that `y` is an essential input.
graph2 = AsynchronousGraph(bn2)
for x in graph2.mk_unit_colors():
    print(x)

ColorModel({'f': '(!x_0 & x_1)'})
ColorModel({'f': '!x_0'})
ColorModel({'f': 'x_1'})
ColorModel({'f': '(!x_0 | (x_0 & x_1))'})
ColorModel({'f': '(x_0 & !x_1)'})
ColorModel({'f': '!x_1'})
ColorModel({'f': 'x_0'})
ColorModel({'f': '((!x_0 & !x_1) | x_0)'})


In [12]:
# We can "transfer" the original set of colors from the initial symbolic graph to the new one.
# We can then use this transferred set as a restriction on the initial conditions of any 
# algorithm/method we want to apply to the reduced network.
old_unit_set_in_new_context = graph2.transfer_from(graph.mk_unit_colors(), graph)
for x in old_unit_set_in_new_context:
    print(x)

ColorModel({'f': '(x_0 & x_1)'})
ColorModel({'f': '((!x_0 & x_1) | x_0)'})


### Working with reduced networks

As long as the network variables share the same names and are ordered the same way, `AsynchronousGraph.transfer_from` should be able to translate symbolic sets between networks. (Default variable ordering is alphabetic, so unless you explicitly change it, the ordering requirement should be satisfied.)

We can use this to implement various algorithms that rely on network reductions. Here is an example of an algorithm that uses reductions to find fixed-points.

In [13]:
from typing import Optional

def reduce(stg: AsynchronousGraph) -> Optional[tuple[AsynchronousGraph, VariableId]]:
    """
    Greedily find the "best" variable to reduce, and return it, including the symbolically
    reduced graph.

    "Best" here means that the variable has the smallest update function in terms of BDD nodes.
    Other greedy criteria could be consider instead.
    """
    best_var = (None, 0)
    for var in stg.network_variables():
        fn_bdd = stg.mk_update_function(var)
        support = fn_bdd.support_set()
        bdd_var = stg.symbolic_context().find_network_bdd_variable(var)
        if bdd_var in support:
            # This variable has a self-dependence.
            continue
            
        if len(fn_bdd) < best_var[1] or best_var[1] == 0:
            best_var = (var, len(fn_bdd))
    if best_var[0] is None:
        return None
    else:
        return (stg.inline_variable_symbolic(best_var[0]), best_var[0])

def fixed_points_rec(stg: AsynchronousGraph) -> ColoredVertexSet:
    """
    Try to greedily reduce the graph/network as much as possible using the symbolic reduction,
    and then recursively extract the set of fixed points.
    """
    reduced = reduce(stg)
    if reduced is None:
        print(f"Search for fixed points in the irreducible network.")
        # This network cannot be reduced further. Use the basic fixed-point search.
        fix = FixedPoints.symbolic(stg, stg.mk_unit_colored_vertices())
        print(f"Found {fix} in the irreducible network.")
        return fix

    (red_stg, red_var) = reduced
    print(f"Reduced network to {red_stg.network_variable_count()} variables.")
    red_fix = fixed_points_rec(red_stg)
    # The variable that is missing in `red_stg` becomes unconstrained when transferred to `stg`.
    # I.e. the states in `red_fix` are extended with both `red_var=0` and `red_var=1`.
    fix_candidates = stg.transfer_from(red_fix, red_stg)
    # Remove any spurious candidates that have a transition under `red_var`.
    fix = fix_candidates.minus(stg.var_can_post(red_var, fix_candidates))
    print(f"Lifted {red_var}, new candidates: {fix}")
    return fix

In [14]:
# A super trivial example with a single fixed-point.
bn = BooleanNetwork.from_file('./data/g2a.sbml')
print(bn)
stg = AsynchronousGraph(bn)

BooleanNetwork(variables=5, regulations=15, explicit_parameters=0, implicit_parameters=0)


In [15]:
fix = fixed_points_rec(stg)
print(f"Found {fix.vertices()} vertices across {fix.colors()} colors (complete relation: {fix}).")
assert fix == FixedPoints.symbolic(stg, stg.mk_unit_colored_vertices()) # Check that the result is correct.

Reduced network to 4 variables.
Reduced network to 3 variables.
Reduced network to 2 variables.
Reduced network to 1 variables.
Reduced network to 0 variables.
Search for fixed points in the irreducible network.
Found ColoredVertexSet(cardinality=1, symbolic_size=2) in the irreducible network.
Lifted v_0, new candidates: ColoredVertexSet(cardinality=1, symbolic_size=3)
Lifted v_1, new candidates: ColoredVertexSet(cardinality=1, symbolic_size=4)
Lifted v_1, new candidates: ColoredVertexSet(cardinality=1, symbolic_size=5)
Lifted v_3, new candidates: ColoredVertexSet(cardinality=1, symbolic_size=6)
Lifted v_3, new candidates: ColoredVertexSet(cardinality=1, symbolic_size=7)
Found VertexSet(cardinality=1, symbolic_size=7) vertices across ColorSet(cardinality=1, symbolic_size=2) colors (complete relation: ColoredVertexSet(cardinality=1, symbolic_size=7)).


In [16]:
# An example with two implicit uninterpreted functions.
bn2 = BooleanNetwork.from_file('./data/g2a_p1026.aeon')
print(bn2)
stg2 = AsynchronousGraph(bn2)

BooleanNetwork(variables=5, regulations=15, explicit_parameters=0, implicit_parameters=2)


In [17]:
fix2 = fixed_points_rec(stg2)
print(f"Found {fix2.vertices()} vertices across {fix2.colors()} colors (complete relation: {fix2}).")
assert fix2 == FixedPoints.symbolic(stg2, stg2.mk_unit_colored_vertices()) # Check that the result is correct.

Reduced network to 4 variables.
Reduced network to 3 variables.
Search for fixed points in the irreducible network.
Found ColoredVertexSet(cardinality=438, symbolic_size=275) in the irreducible network.
Lifted v_3, new candidates: ColoredVertexSet(cardinality=438, symbolic_size=314)
Lifted v_3, new candidates: ColoredVertexSet(cardinality=438, symbolic_size=316)
Found VertexSet(cardinality=3, symbolic_size=11) vertices across ColorSet(cardinality=417, symbolic_size=268) colors (complete relation: ColoredVertexSet(cardinality=438, symbolic_size=316)).


In [18]:
# A larger network with 13 input parameters where a lot of the variables can be actually reduced.
bn3 = BooleanNetwork.from_file('./data/butanol-pathway.sbml')
bn3 = bn3.inline_inputs(infer_inputs=True, repair_graph=True)
print(bn3)
stg3 = AsynchronousGraph(bn3)

BooleanNetwork(variables=53, regulations=117, explicit_parameters=13, implicit_parameters=0)


In [19]:
fix3 = fixed_points_rec(stg3)
print(f"Found {fix3.vertices()} vertices across {fix3.colors()} colors (complete relation: {fix3}).")
assert fix3 == FixedPoints.symbolic(stg3, stg3.mk_unit_colored_vertices())  # Check that the result is correct.

Reduced network to 52 variables.
Reduced network to 51 variables.
Reduced network to 50 variables.
Reduced network to 49 variables.
Reduced network to 48 variables.
Reduced network to 47 variables.
Reduced network to 46 variables.
Reduced network to 45 variables.
Reduced network to 44 variables.
Reduced network to 43 variables.
Reduced network to 42 variables.
Reduced network to 41 variables.
Reduced network to 40 variables.
Reduced network to 39 variables.
Reduced network to 38 variables.
Reduced network to 37 variables.
Reduced network to 36 variables.
Reduced network to 35 variables.
Reduced network to 34 variables.
Reduced network to 33 variables.
Reduced network to 32 variables.
Reduced network to 31 variables.
Reduced network to 30 variables.
Reduced network to 29 variables.
Reduced network to 28 variables.
Reduced network to 27 variables.
Reduced network to 26 variables.
Reduced network to 25 variables.
Reduced network to 24 variables.
Reduced network to 23 variables.
Reduced ne

In [20]:
import time

start = time.perf_counter_ns()
fix_basic = FixedPoints.symbolic(stg3, stg3.mk_unit_colored_vertices())
elapsed_basic = time.perf_counter_ns() - start

start = time.perf_counter_ns()
fix_reduced = fixed_points_rec(stg3)
elapsed_reduced = time.perf_counter_ns() - start

Reduced network to 52 variables.
Reduced network to 51 variables.
Reduced network to 50 variables.
Reduced network to 49 variables.
Reduced network to 48 variables.
Reduced network to 47 variables.
Reduced network to 46 variables.
Reduced network to 45 variables.
Reduced network to 44 variables.
Reduced network to 43 variables.
Reduced network to 42 variables.
Reduced network to 41 variables.
Reduced network to 40 variables.
Reduced network to 39 variables.
Reduced network to 38 variables.
Reduced network to 37 variables.
Reduced network to 36 variables.
Reduced network to 35 variables.
Reduced network to 34 variables.
Reduced network to 33 variables.
Reduced network to 32 variables.
Reduced network to 31 variables.
Reduced network to 30 variables.
Reduced network to 29 variables.
Reduced network to 28 variables.
Reduced network to 27 variables.
Reduced network to 26 variables.
Reduced network to 25 variables.
Reduced network to 24 variables.
Reduced network to 23 variables.
Reduced ne

In [21]:
print(elapsed_basic, elapsed_reduced, "; speedup", elapsed_basic / elapsed_reduced)

4602333 5099333 ; speedup 0.9025362728811788


Note that in this simple case, the difference in runtime is measurable, but not substantial, since both methods effectively require milliseconds to complete. In fact, depending on your hardware and run-to-run variance, the reduction method can be even slower.

However, on larger or more complex networks, the reduction method can be much faster than the standard one (do take these numbers with a grain of salt though; a proper benchmark should at the very least average multiple runs or consider more than a handful of models):

In [22]:
bn4 = BooleanNetwork.from_file('./data/random.bnet')
print(bn4)
stg4 = AsynchronousGraph(bn4)

BooleanNetwork(variables=100, regulations=200, explicit_parameters=0, implicit_parameters=0)


In [23]:
start = time.perf_counter_ns()
fix_basic = FixedPoints.symbolic(stg4, stg4.mk_unit_colored_vertices())
elapsed_basic = time.perf_counter_ns() - start

start = time.perf_counter_ns()
fix_reduced = fixed_points_rec(stg4)
elapsed_reduced = time.perf_counter_ns() - start

Reduced network to 99 variables.
Reduced network to 98 variables.
Reduced network to 97 variables.
Reduced network to 96 variables.
Reduced network to 95 variables.
Reduced network to 94 variables.
Reduced network to 93 variables.
Reduced network to 92 variables.
Reduced network to 91 variables.
Reduced network to 90 variables.
Reduced network to 89 variables.
Reduced network to 88 variables.
Reduced network to 87 variables.
Reduced network to 86 variables.
Reduced network to 85 variables.
Reduced network to 84 variables.
Reduced network to 83 variables.
Reduced network to 82 variables.
Reduced network to 81 variables.
Reduced network to 80 variables.
Reduced network to 79 variables.
Reduced network to 78 variables.
Reduced network to 77 variables.
Reduced network to 76 variables.
Reduced network to 75 variables.
Reduced network to 74 variables.
Reduced network to 73 variables.
Reduced network to 72 variables.
Reduced network to 71 variables.
Reduced network to 70 variables.
Reduced ne

In [24]:
assert fix_basic == fix_reduced
print(fix_basic)
print(elapsed_basic, elapsed_reduced, "speedup", elapsed_basic / elapsed_reduced)

ColoredVertexSet(cardinality=2, symbolic_size=191)
611757875 14662500 speedup 41.72261722080136
